
# Inertia as a Form of Model Compression in Convolutional Neural Networks


> *In submission for Deep Learning Project, Spring 2025*

**Labiba Shahab**

**Ahmed Wali**


> *What you usually achieve with a standard convolution, you can accomplish efficiently using a smaller convolution + peripheral inertia mechanism (inertial filter)*
~ Group 39


## In a Gist:

Standard dxd convolution layers involve d^2 learnable parameters and d^2 computations per convolution operation. In this project, we propose an inertial convolution mechanism that dynamically decides whether a detailed convolution is necessary. The goal is to reduce both computations and learnable parameters while maintaining performance on vision tasks like MNIST classification.

Instead of learning a full dxd kernel, we use a (d-k)x(d-k) core filter to convolve a central patch. The surrounding d^2-(d-k)^2 pixels act as an inertial periphery—evaluating local divergence or “friction.” If the divergence is high, we re-apply the core filter across the full dxd region in another convolution and stack the outputs. If low, we skip detailed computation.

In the best case, we perform just d-k computation with d-k learnable parameters.
In the worst case, we perform up to d computations, but still only learn d-k parameters, hence effectively pruning the model with estimation compression.

## Background

There has been extensive research in reducing neural network complexity:

Dynamic convolutions and skip-convolutions conditionally skip expensive operations. This project is inspired from [Dynamic Sparse Convolutions](https://arxiv.org/pdf/2102.04906), [Skip Convolutions](https://openaccess.thecvf.com/content/CVPR2021/papers/Habibian_Skip-Convolutions_for_Efficient_Video_Processing_CVPR_2021_paper.pdf), and [Fractional Skipping](https://arxiv.org/abs/2001.00705)

Pruning, quantization, and knowledge distillation reduce parameters, memory, or model depth.

Our approach is inspired by these ideas but focuses on parameter reuse and friction-aware skipping, offering a novel trade-off between learning capacity and computational efficiency.

#### Novelty

Inspired from computational optimization, our project proposes a similar mechanism to do model compression by estimation.

## Baseline Models

#### Fashion MNIST
The baseline CNN architecture is adapted from the official [Fashion-MNIST GitHub benchmark](https://github.com/zalandoresearch/fashion-mnist) implementation in TensorFlow. We replicated the structure and hyperparameters in PyTorch for fair benchmarking.

Reproducibility + Imports

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
import random
import numpy as np

Seeds and CuDNN configs for deterministic results on a GPU

In [2]:
seed = 1
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Load Fashion MNIST Dataset

In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.FashionMNIST('./data', train=True, download=True, transform=transform)
test_dataset = datasets.FashionMNIST('./data', train=False, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=400, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1000, shuffle=False)

100%|██████████| 26.4M/26.4M [00:02<00:00, 10.1MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 164kB/s]
100%|██████████| 4.42M/4.42M [00:01<00:00, 3.19MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 22.6MB/s]


Test Function

In [4]:
def test(model, device, test_loader):
    model.eval()
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    accuracy = correct / len(test_loader.dataset)
    print(f"Accuracy: {accuracy:.10f}")
    return accuracy

#### Fashion MNIST 5x5 ConvNet Model


In [ ]:
#@originally adapted from https://github.com/zalandoresearch/fashion-mnist/blob/master/benchmark/convnet.py
class FashionNet5x5(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 5, padding=2)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 5, padding=2)
        self.fc1 = nn.Linear(7*7*64, 1024)
        self.dropout = nn.Dropout(0.4)
        self.fc2 = nn.Linear(1024, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

#### Fashion MNIST 3x3 ConvNet Model


In [6]:
# @adaptation of the above 5x5 model to use 3x3 convolutions
class FashionNet3x3(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.fc1 = nn.Linear(7 * 7 * 64, 1024)
        self.dropout = nn.Dropout(0.4)
        self.fc2 = nn.Linear(1024, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

#### Adapted Fashion MNIST 1x1 ConvNet Model

In [7]:
#@adaptation of the above 5x5 model to use 1x1 convolutions
class FashionNet1x1(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 1)
        self.fc1 = nn.Linear(7*7*64, 1024)
        self.dropout = nn.Dropout(0.4)
        self.fc2 = nn.Linear(1024, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

Training Function

In [8]:
def train_model(model, name):
    model = model.to(device)
    optimizer = optim.SGD(model.parameters(), lr=0.001)
    epochs = 100

    for epoch in range(epochs):
        model.train()
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = F.cross_entropy(output, target)
            loss.backward()
            optimizer.step()

    print(f"\n{name} Results:")
    acc = test(model, device, test_loader)
    return acc

So Let's Evaluate Fashion MNIST on the Baseline

In [9]:
model_5x5 = FashionNet5x5()
acc_5x5 = train_model(model_5x5, "FashionNet 5x5")

model_3x3 = FashionNet3x3()
acc_3x3 = train_model(model_3x3, "FashionNet 3x3")

model_1x1 = FashionNet1x1()
acc_1x1 = train_model(model_1x1, "FashionNet 1x1")


FashionNet 5x5 Results:
Accuracy: 0.8338000000

FashionNet 3x3 Results:
Accuracy: 0.8317000000

FashionNet 1x1 Results:
Accuracy: 0.8169000000


#### Note:

Fashion-MNIST is a drop-in replacement for MNIST but contains Zalando's article images.
- **Existing SOA Model**: Simple CNN / ResNet-based
- **Expected Accuracy**: ~93-95%
- **Link**: [Fashion-MNIST GitHub](https://github.com/zalandoresearch/fashion-mnist)

This gives a little more divergence, hence giving us with a more naunced understanding of the application of the inertial filters.